# Map auth to mmcif numbering
Map AUTH to sequences. Extract the sequences as well.
### WARNING: 
here we run the enhancement using the AHoJ-DB (see branch [scPDB_enhancement in CryptoBench](https://github.com/skrhakv/CryptoBench/tree/scPDB_enhancement))

Also, you need to manually change `4abzA` to `4abzAAA` in the `scPDB_enhanced_binding_sites.csv`!

In [ ]:
# TODO: take code from visualize.ipynb and adapt here
import sys
import numpy as np

DATA_DIRECTORY = '/home/vit/Projects/cryptoshow-analysis/data/data-extraction'
DATA_PATH = f'{DATA_DIRECTORY}/scPDB_enhanced_binding_sites.csv'

sys.path.append('/home/vit/Projects/cryptoshow-analysis/src/utils')
import vis_utils
import eval_utils
import cryptoshow_utils
import csv

def read_test_binding_residues(data_path=DATA_PATH, pocket_types=['CRYPTIC']) -> set[int]:
    cryptic_binding_residues = {}

    with open(data_path, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=';')
        for row in reader:
            chain_id = row[1]
            pdb_id = row[0]
            protein_id = f'{pdb_id}{chain_id}'

            if row[3] == '':
                continue

            binding_residue_indices = [f'{chain_id}_{i}'for i in row[3].split(' ')]
            if row[2] in pocket_types:
                if protein_id not in cryptic_binding_residues:
                    cryptic_binding_residues[protein_id] = []
                cryptic_binding_residues[protein_id].append(binding_residue_indices)

    return cryptic_binding_residues

def reformat_binding_residues(binding_residues: dict) -> dict:
    reformated = {}
    for protein_id, residues in binding_residues.items():
        assert len(residues) == 1, "Expected only one pocket per protein in scPDB"
        reformated_protein_id = protein_id.replace('_', '')
        reformated_residues = np.array([str(''.join(filter(str.isdigit, residue.split('_')[1]))) for residue in residues[0]])
        reformated[reformated_protein_id] = reformated_residues
    return reformated

# load ground truth binding residues: these have mmcif numbering and need to be mapped to auth labeling
binding_residues_mmcifed = read_test_binding_residues(data_path=DATA_PATH, pocket_types=['NON_CRYPTIC'])
binding_residues_mmcifed = reformat_binding_residues(binding_residues_mmcifed)


In [ ]:
with open(f"{DATA_DIRECTORY}/scPDB_enhanced_binding_sites_translated.csv", 'w') as f:
    for protein_id, pocket in binding_residues_mmcifed.items():
        binding_residues_mapped, sequence = cryptoshow_utils.map_auth_to_mmcif_numbering(protein_id[:4], protein_id[4:], pocket)
        line = f'{protein_id[:4]};{protein_id[4:]};UNKNOWN;{" ".join([str(i) for i in binding_residues_mapped])};{sequence}\n'
        f.write(line)

In [ ]:
for protein_id in binding_residues_mmcifed.keys():
    coordinates = cryptoshow_utils.get_coordinates(protein_id[:4], protein_id[4:])
    np.save(f'{DATA_DIRECTORY}/enhanced-scPDB-coordinates/{protein_id}.npy', coordinates)